In [1]:
import torch
import pickle
import time
import numpy as np
import matplotlib.pyplot as plt
import utils.inverter as inv
from models.stylegan_generator_idinvert import StyleGANGeneratorIdinvert

In [2]:
# load real images
path_images = 'runners/losses/fourier/stylegan_idinver_png/'

original_list = []

for ii in range(7):
    original_list.append(plt.imread(path_images+str(ii)+'/original.png'))

original = np.array(original_list)
original = original[:,:,:,:3]
original_transform = original * 2 - 1

In [3]:
# initialize generator & invertor
G = StyleGANGeneratorIdinvert('styleganinv_ffhq256')
Inverter = inv.StyleGANInverter(G, 'styleganinv_ffhq256', iteration=10)

[2020-12-23 19:46:18,156][INFO] Build network for module `generator` in model `styleganinv_ffhq256`.
[2020-12-23 19:46:18,384][INFO] Loading pytorch weights from `models/pretrain/styleganinv_ffhq256_generator.pth`.
[2020-12-23 19:46:18,778][INFO] Successfully loaded!
[2020-12-23 19:46:18,793][INFO] Current `lod` is 0.0.
[2020-12-23 19:46:18,794][INFO] Build network for module `encoder` in model `styleganinv_ffhq256`.
[2020-12-23 19:46:19,983][INFO] Loading pytorch weights from `models/pretrain/styleganinv_ffhq256_encoder.pth`.
[2020-12-23 19:46:22,006][INFO] Successfully loaded!


In [4]:
# create optimized latent code & real images
start = time.time()

result = []
for ii in range(original_transform.shape[0]):
    toInvert = torch.from_numpy(original_transform[ii]).permute(2,0,1).unsqueeze(0)
    result.append(Inverter.invert_offline(image=toInvert))
    
print((time.time()-start)/60, 'min')
print('=',(time.time()-start)/3600, 'h')

loss_pix: 0.020, loss_feat: 2979.996, loss_reg: 0.015, loss: 0.200: 100%|██████████| 10/10 [01:22<00:00,  8.29s/it]
loss_pix: 0.018, loss_feat: 2402.664, loss_reg: 0.011, loss: 0.160: 100%|██████████| 10/10 [01:23<00:00,  8.34s/it]
loss_pix: 0.120, loss_feat: 8796.975, loss_reg: 0.041, loss: 0.641: 100%|██████████| 10/10 [01:23<00:00,  8.36s/it]
loss_pix: 0.029, loss_feat: 3643.846, loss_reg: 0.022, loss: 0.254: 100%|██████████| 10/10 [01:23<00:00,  8.33s/it]
loss_pix: 0.016, loss_feat: 1981.657, loss_reg: 0.010, loss: 0.134: 100%|██████████| 10/10 [01:23<00:00,  8.31s/it]
loss_pix: 0.101, loss_feat: 3459.869, loss_reg: 0.172, loss: 0.617: 100%|██████████| 10/10 [01:23<00:00,  8.35s/it]
loss_pix: 0.074, loss_feat: 3621.046, loss_reg: 0.046, loss: 0.346: 100%|██████████| 10/10 [01:23<00:00,  8.35s/it]


10.024081834157307 min
= 0.16706809474362266 h


In [12]:
# save
pickle.dump(result, open( "res.p", "wb" ))